# Study of Burgers's equation in 1D

## Physical model
As physical model we use Burgers equation.
This equation is non-linear and can lead to interesting shock formations. Hence, it's a very good starting point for experiments, and it's 1D version is given by:

$$
    \frac{\partial u}{\partial{t}} + u \nabla u =
    \nu \nabla\cdot \nabla u
$$ 


Preliminaries: first we import the phiflow library to generate data.

In [ ]:
!pip install --quiet phiflow  # Install stable release


In [ ]:
import pylab
from phi.torch.flow import *


Let define and initialize the necessary constants:
our simulation domain will have `N=128` cells as discretization points for the 1D velocity $u$ in a periodic domain $\Omega$ for the interval $[-1,1]$, using 32 time `STEPS` for a time interval of 1, giving us `DT=1/32`. Additionally,  viscosity `NU` is set to $\nu=0.01/\pi$.

The initial state is given by $-\text{sin}(\pi x)$ in the numpy array `INITIAL_NUMPY`, which will be used to initialize the velocity $u$ in the simulation in the next cell. This initialization will produce a nice shock in the center of the domain. 

In [ ]:
N = 128
STEPS = 32
DT = 1./STEPS
NU = 0.01/np.pi

# initialization of velocities
INITIAL_NUMPY = np.asarray( [-np.sin(np.pi * x) * 1. for x in np.linspace(-1,1,N)] ) # 1D numpy array
INITIAL = math.tensor(INITIAL_NUMPY, spatial('x') ) # convert to phiflow tensor
velocity = CenteredGrid(INITIAL, extrapolation.PERIODIC, x=N, bounds=Box[-1:1])
velocities = [velocity]
age = 0.
for i in range(STEPS):
    v1 = diffuse.explicit(velocities[-1], NU, DT)
    v2 = advect.semi_lagrangian(v1, v1, DT)
    age += DT
    velocities.append(v2)
# get "velocity.values" from each phiflow state with a channel dimensions, i.e. "vector"
vels = [v.values.numpy('x,vector') for v in velocities] # gives a list of 2D arrays 

Once the simulation is performed, let's convert the data into PyTorch tensors. $x$ is the spatial domain, $t$ is the time, $y$ is the velocity computed from the simulation. In a supervised learning framework, we need triplet values $(x_i, t_i, y_i)$ where $i$ denotes a sample.


In [ ]:
import numpy as np

vels = [v.values.numpy('x,vector') for v in velocities] # gives a list of 2D arrays 
x = torch.reshape(torch.linspace(-1,1,128),(-1,1))
t = torch.reshape(torch.linspace(0,1,33),(-1,1))
x = x.repeat(33,1)
t = t.repeat_interleave(128, dim=0)
v = np.array(vels).reshape(-1,1)
y = torch.from_numpy(v)  

def plot_1d(y_vis):
  fig = pylab.figure().gca()
  fig.plot(y_vis[0,:], color='blue',  label="t=0")
  fig.plot(y_vis[10,:], color='green', label="t=0.3125")
  fig.plot(y_vis[20,:], color='cyan',  label="t=0.625")
  fig.plot(y_vis[32,:], color='purple',label="t=1")
  pylab.xlabel('x'); pylab.ylabel('u'); pylab.legend()  

plot_1d(y.reshape((33,128)))  

The 1D display shows the shock developing in the center of the domain, which forms from the collision of the two initial velocity "bumps", the positive one on left (moving right) and the negative one right of the center (moving left).

An alternative approach to visualize the velocities is by displaying an image (the vertical axis is the spatial domain while the other axis is the "dilated" time).

In [ ]:
def plot_2d(y_vis):
  fig, axes = pylab.subplots(1, 1, figsize=(16, 5))
  im = axes.imshow(y_vis, origin='upper', cmap='inferno')
  pylab.colorbar(im) ; pylab.xlabel('time'); pylab.ylabel('x'); 

plot_2d(np.transpose(y.reshape(33,128)).repeat_interleave(16, dim=1))


## Supervised learning
Let's consider a reconstruction (or interpolation) task:
$
\text{arg min}_{\theta} \sum_i ( f(x_i ; \theta)-y_i )^2 
$
where $x$ and $y$ are solutions of $u$ at different locations in space and time. Note that there is no constraint for the boundary conditions.

First, set up a neural network that will be used to approximate the velocities, define appropriate loss function and optimizer.

In [ ]:
import torch.nn as nn
class burgers_net(nn.Module):
    
    # class initialization
    def __init__(self, input_size, hidden_size, n_layers, output_size):
        super(burgers_net, self).__init__()
        
        self.n_layers = n_layers
        self.fc0 = nn.Linear(input_size, hidden_size)
        self.fcs = torch.nn.ModuleList()
        for i in range(self.n_layers):
          self.fcs.append(nn.Linear(hidden_size, hidden_size))
        self.fcn = nn.Linear(hidden_size, output_size)
        
    # function to apply the neural network
    def forward(self, x, t):
        xt = torch.cat((x,t),dim=1)
        y = self.fc0(xt)
        y = nn.Tanh()(y)
        for i in range(self.n_layers):
          y = self.fcs[i](y)
          y = nn.Tanh()(y)
        y_pred = self.fcn(y)  
        return y_pred

In [ ]:
net_model = burgers_net(input_size = 2,hidden_size = 20, n_layers = 2, output_size = 1)

In [ ]:
import torch.optim as optim
mse = nn.MSELoss()
optimizer = torch.optim.Adam(net_model.parameters(), lr = 0.0001)

Start the training step ...

In [ ]:
%matplotlib inline
%pylab inline

epochs = 10000 # number of epochs
losses = [] # list to stock the loss at each iteration

# Loop on epochs
for i in range(epochs):
    
    # compute the prediction using the previous parameters of the neural network
    y_pred = net_model.forward(x,t)
    
    # compute and stock the loss
    loss = mse(y_pred, y)
    losses.append(loss)
    
    # initialize the gradient to zero
    optimizer.zero_grad()
    
    # compute the gradient by back propagation
    loss.backward()
    
    # update the parameter values using the gradient
    optimizer.step()

plot(range(epochs), losses)
title('Loss function', size=20)
xlabel('Epoch', size=20)
ylabel('Loss value', size=20)

In [ ]:
# plot the adjustment at the last epoch
plot(x[0:128,0], y[0:128,0], 'r.')
plot(x[0:128,0], y_pred.detach().numpy()[0:128,0], 'g.')
legend(['Simulation', 'Fit'], prop={'size': 20})
title('Nonlinear regression', size=20)
xlabel('x', size=20)
ylabel('y', size=20)

We can plot the estimated velocities using the ```plot_1d``` function.

In [ ]:
y_vis = y_pred.reshape((33,128)).detach().numpy()
plot_1d(y_vis)  

We can also plot the error between the estimated velocities and the simulated values using the ```plot_1d``` function. For quantitative purpose, we can compute the mean absolute error.

In [ ]:
y_true = y.reshape((33,128))

plot_1d(y_true - y_vis)

print(np.mean(np.abs(y_true.numpy()-y_vis).flatten()))

In [ ]:
fig = pylab.figure().gca()
fig.plot(y_vis[32,:], color='purple',label="t=1")
fig.plot(y_true[32,:], color='blue',label="t=1")
pylab.xlabel('x'); pylab.ylabel('u'); pylab.legend()

In [ ]:
y_vis = torch.transpose(y_pred.reshape(33,128),0,1).repeat_interleave(16, dim=1).detach().numpy()
plot_2d(y_vis)

## Subsampling

Let's consider now a similar supervised learning problem with subsampled data. We only keep 200 points for the training step.

In [ ]:
idx = torch.randperm(x.shape[0])
n_samples = 200
subx = x[idx[0:n_samples],:]
subt = t[idx[0:n_samples],:]
suby = y[idx[0:n_samples],:]

net_model_2 = burgers_net(input_size = 2,hidden_size = 20, n_layers = 2, output_size = 1)
optimizer = torch.optim.Adam(net_model_2.parameters(), lr = 0.0001)

epochs = 10000 # number of epochs
# Loop on epochs
for i in range(epochs):
    y_pred = net_model_2.forward(subx,subt)
    loss = mse(y_pred, suby)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

y_pred = net_model_2.forward(x,t)
y_vis = y_pred.reshape((33,128)).detach().numpy()

plot_1d(y_vis)

print(np.mean(np.abs(y_true.numpy()-y_vis).flatten()))

## Physically informed deep learning

We consider now the following reconstruction problem:

$
\text{arg min}_{\theta} \sum_i ( f(x_i ; \theta)-y_i )^2 + R(f(x_i ; \theta)) ,
$

The residual function $R$ collects additional evaluations of $f(;\theta)$ and its derivatives to formulate the residual of the physical modeling. This approach -- using derivatives of a neural network to compute a PDE residual -- is typically called a _physics-informed_ approach, yielding a _physics-informed neural network_ (PINN) to represent a solution for the inverse reconstruction problem.

In the formulation above, $R$ should simply converge to zero above. 

Start to write the function for the computation of $R$.

In [ ]:
#############
### TO DO ###
#############

def residual_function(u,x,t):

    ...
    
    return u_t + u*u_x - (0.01 / np.pi) * u_xx

Perform the training using the new losses (data-fidelity and regularization terms).

In [ ]:
#############
### TO DO ###
#############

...

# Loop on epochs
for i in range(epochs):
    
    ...

Plot the estimated velocities, as well as the error between the predicted values and the simulated values.

In [ ]:
#############
### TO DO ###
#############